## Clustering draft 1

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the module YahooFinance 

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier


We also import the module1.py file to use the function we defined there.

In [2]:
pip install git+https://github.com/robertmartin8/PyPortfolioOpt.git

  Cloning https://github.com/robertmartin8/PyPortfolioOpt.git to /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-krqwow0m
  Running command git clone --filter=blob:none --quiet https://github.com/robertmartin8/PyPortfolioOpt.git /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-krqwow0m
  Resolved https://github.com/robertmartin8/PyPortfolioOpt.git to commit 30ab57147ba61eddc8301294a5b1c5ef260b23fa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

#### 1.1. Scraping the names of the assets of interest

- This is important because to import financial data for assets on Yahoo Finance, you need the tickers' names. This scraping work was performed in the file *get_S&P500_symbols.ipynb* and generated the *S&P500_symbols.csv* file.

- We rely on the csv package to read the *S&P500_symbols.csv* file and to create a list containing all the tickers

In [4]:
import csv 

with open('S&P500_symbols.csv', mode='r') as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    
    # Transform the CSV data into a list
    stock_symbols = []
    for row in csv_reader:
        stock_symbols.append(row[0])

# Close the file
file.close()

stock_symbols.pop(0)

'Symbols of S&P 500'

In [5]:
stock_symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADM',
 'ADBE',
 'ADP',
 'AES',
 'AFL',
 'A',
 'ABNB',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',


#### 1.2. Creating the dataframe using pandas and yfinance 

In this case, we focus on the financial data of asset during the year 2022

In [6]:
n_stocks = len(stock_symbols) # number of stocks = 502

start = "2022-01-01" # start date
end = "2022-12-01" # end date

df = pd.DataFrame(yf.download(stock_symbols, start, end)) # data on the 198 assets
data = np.log(df["Close"]/df["Open"]).transpose() # compute the returns of these assets
data = data.dropna()

[*********************100%%**********************]  502 of 502 completed


5 Failed downloads:
['GEHC', 'VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1669870800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-01)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Due to missing values in the data, 5 stocks have been excluded from the intial 502 we considered. Thus the shape of the dataframe is (496, 230) 






### 2. We repeat the clustering step several times to start preparing multiple micro-portfolios

**General idea**: The multiple_clustering function takes as arguments:

1. *n_repeat*: an integer that corresponds to the number of times we want to train (.fit()) the model passed as an argument on the data.

2. *data*: a pandas dataframe (like the one we generated above) that corresponds to the financial data on which the model will be trained.

3. *model*: a sklearn clustering model (for now, we will test it with KMeans).

4. *model_name*: the name of the model, necessary for creating the pipeline properly.

**Outcome**: a pandas dataframe storing the results of the multiple clustering and a dictionary containing the centroids of the clusters each time 

We first make the test using the K-Means model and fix we fix the number of clusters to 5

In [7]:
model = KMeans(n_clusters=5)
model_name = 'kmeans'

Y, C = module1.multiple_clusterings(10, data, model, model_name)
print(Y, C)

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

      Clustering n°1  Clustering n°2  Clustering n°3  Clustering n°4  \
A                  3               0               2               4   
AAL                2               4               1               0   
AAPL               0               0               2               4   
ABBV               1               3               3               1   
ABNB               2               1               1               0   
...              ...             ...             ...             ...   
YUM                1               0               2               4   
ZBH                3               0               2               4   
ZBRA               0               0               4               3   
ZION               3               4               2               4   
ZTS                1               0               2               4   

      Clustering n°5  Clustering n°6  Clustering n°7  Clustering n°8  \
A                  0               2               3           

In [8]:
Y

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
A,3,0,2,4,0,2,3,2,2,1
AAL,2,4,1,0,3,4,4,4,1,3
AAPL,0,0,2,4,0,2,3,1,2,1
ABBV,1,3,3,1,2,1,1,3,4,4
ABNB,2,1,1,0,3,4,0,4,1,3
...,...,...,...,...,...,...,...,...,...,...
YUM,1,0,2,4,0,2,3,3,2,1
ZBH,3,0,2,4,0,2,3,2,2,1
ZBRA,0,0,4,3,4,2,0,1,0,3
ZION,3,4,2,4,0,2,4,2,2,2


### 3. Compute the weights of the stocks in each cluster 

#### 3.1. Get the cluster compositions each time 

**Why doing so?**: the purpose of this work is to compute the returns of each cluster (seen as a new "fictive" asset composed of "real" assets) and this latter task is easier if we know the cluster composition in terms of tickers (because in the financial database we have data given a ticker).

In [10]:
Y_symbol = module1.cluster_composition(Y)
Y_symbol

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[A, ACN, AES, AIG, AKAM, ALLE, AME, AMP, AOS, ...","[A, AAPL, ABT, ACN, ADP, AES, AKAM, ALLE, AME,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AJG, AKAM, ALLE,...","[A, ACN, AES, AIG, ALLE, AME, AMP, AOS, APD, A...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADI, ADP, AES, AJG, AKAM, ..."
Cluster 2,"[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, BX, CCL...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ABNB, ALK, BA, BBWI, CCL, CZR, DAL, EXPE...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ADBE, ADSK, ALB, ALGN, AMAT, AMD, ..."
Cluster 3,"[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMC...","[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMC..."
Cluster 4,"[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ...","[ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[AIG, ALK, AMP, AVY, AXP, BA, BAC, BALL, BBY, ..."
Cluster 5,"[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ..."


#### 3.2. Compute the weights of each assets within a cluster

**Why doing so?**: because each stock in a cluster has a different weight. Stock are "representative" of the cluster in which they live to a different extent. So far, we built the weight of a stock based on its $\mathcal{L}^2$ distance from the cluster's centroid. We can therefore see the weights as a kind of "inertia contribution" of the stock in the cluster.

##### 3.2.1. Analysis of the centroids 

- The .multiple_clusterings() function returns two data frame, the first of which is a dataframe containing the centroid of each cluster. This centroid is an array containing the "average" return each day, where each daily average is taken on the returns of all the stocks contained in the corresponding cluster. 

- Here is an example:


In [11]:
C

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[-0.28931492711993867, -1.165199873872467, -1....","[-0.5865867773383071, -0.22983365349887278, -0...","[1.5002065476511288, 1.0570143644422916, -0.31...","[1.0818821358026836, -0.10293663224269473, -0....","[-0.20622988081621363, 0.18987281309778561, 0....","[-0.0625870737441699, -1.1188149295966316, -1....","[-0.04314403662613201, -1.141064652478645, -1....","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.17409859643615738, -1.138527299010478, -1....","[1.5002065476511288, 1.0570143644422916, -0.31..."
Cluster 2,"[-0.12002057436487629, -0.09551460855918219, 0...","[-0.03255987985456288, -1.1632508983716943, -1...","[1.1244838244132374, -0.2749346042866737, -0.8...","[-0.026318604658893043, -0.02019278990310089, ...","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.01922149164208389, 0.005907972939479033, 0...","[0.014131603021133709, 0.04997798943790437, 0....","[-0.2629261945673874, -1.1214478871371363, -1....","[1.0818821358026836, -0.10293663224269473, -0....","[-0.6675805391424525, -0.3919186963852453, -0...."
Cluster 3,"[1.0818821358026836, -0.10293663224269473, -0....","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.1998535862256062, 0.20554509331052195, 0.0...","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.05479170637683879, -0.022171875492677098, ...","[-0.24729138131829875, 0.13840627718909984, 0....","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.06571036939383815, 0.4200065437233978, 0.0...","[-0.22232290767337284, 0.16625200781813876, 0....","[0.34359472886626896, 0.7059938827024812, 0.21..."
Cluster 4,"[-0.1280218009998571, 0.3798172936321998, 0.07...","[-0.018132911459820412, 0.03899259483432386, 0...","[-0.05479170637683879, -0.022171875492677098, ...","[-0.25594876226768776, -1.1625746749393877, -1...","[1.0818821358026836, -0.10293663224269473, -0....","[1.5002065476511288, 1.0570143644422916, -0.31...","[-0.59847261240854, -0.2636300224195542, -0.12...","[-0.16198005090869663, -0.09213043715637585, 0...","[1.5002065476511288, 1.0570143644422916, -0.31...","[0.15107951287336335, -1.0533078875460355, -1...."
Cluster 5,"[1.5002065476511288, 1.0570143644422916, -0.31...","[0.5970556345871619, 0.677980850398963, 0.0365...","[-0.17409859643615738, -1.138527299010478, -1....","[-0.19533452532576, 0.21851382440422537, 0.050...","[-0.17409859643615738, -1.138527299010478, -1....","[0.9701215787620638, -0.1002422928882306, -0.7...","[0.515186315428237, 0.6473615012811185, 0.0112...","[1.460164669175695, -0.48825906489785076, -0.9...","[-0.01922149164208389, 0.005907972939479033, 0...","[-0.0013111296273393574, 0.026441038066927776,..."


##### 3.2.2. Computing the weights of each cluster

- In module1, there are two functions *cluster_weights* and *gaussian_weights* which, given one cluster and its centroids, return the weights of the stocks in the cluster. These weights corresponds to a sort of distance of the stocks to the cluster's centroid.

- If we consider the previous example, we can consider the first cluster (Cluster 1 of the Clusering n°1) and compute the weights of the stocks in this cluster. 


In [12]:
Y_symbol.head(2)

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[A, ACN, AES, AIG, AKAM, ALLE, AME, AMP, AOS, ...","[A, AAPL, ABT, ACN, ADP, AES, AKAM, ALLE, AME,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AJG, AKAM, ALLE,...","[A, ACN, AES, AIG, ALLE, AME, AMP, AOS, APD, A...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADI, ADP, AES, AJG, AKAM, ..."
Cluster 2,"[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, BX, CCL...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ABNB, ALK, BA, BBWI, CCL, CZR, DAL, EXPE...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ADBE, ADSK, ALB, ALGN, AMAT, AMD, ..."


#### **Example**: 
- To take a basic example, we consider the first cluster of the first clustering (if we refer to the Y_symbol cell above this corresponds to *Y_symbol.iloc[0, 0]* for the tickers and to *C.iloc[0, 0]* for the centroids).

- We then compute the $\mathcal{L}^2$ weights and the Gaussian weights for this cluster 



In [13]:
    ## We consider the first cluster of the first clustering 
    #  (location [0, 0])

cluster = Y_symbol.iloc[0,0]
centroid = C.iloc[0,0]

    ## We then compute the weights (L2 and Gaussian) corresponding to 
    #  the stocks in this cluster 

weights_L2 = module1.cluster_weights(cluster, centroid, data)
weights_gaussian = module1.gaussian_weights(cluster, centroid, data)


In [18]:
cluster_composition = cluster 
micro_portfolio_return = pd.DataFrame(index=cluster_composition, columns=data.columns).transpose()
micro_portfolio_return

,A,ACN,AES,AIG,AKAM,ALLE,AME,AMP,AOS,APD,...,WDC,WFC,WHR,WRK,WTW,WY,XRAY,XYL,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- We make sure that the weights add up to one

In [13]:
print(weights_L2.sum(axis=1))
print(weights_gaussian.sum(axis=1))

0    1.0
dtype: float64
0    1.0
dtype: float64


### 4. Compute the returns of each portfolio, 

- **Idead**: We considered that one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) [*in this notebook, we first fix the number of clusters to be equal to 5*]

To have an idea of how we built the clustering_return() function, see the shape of cluter_data (which corresponds to the return data associated with the tickers in the cluster) and the weights of the stocks in this cluster 

In [20]:
cluster_data = data.loc[cluster]
print(cluster_data.shape)
print(weights_gaussian.shape)

(197, 230)
(1, 197)


#### **Remarks**:

- We see that the to get the return of each clusters, we have to multiply each columns of the cluster_data dataframe by the weights of the stocks (take the transpose of the line dataframe).

- The **clustering_return** function applies to a **clustering** i.e. one column of the Y_symbol dataframe (result of the cluster_composition() funciton). 

In [31]:
## We take as an example the first clustering of the Y_symbol dataframe
clustering_composition_1 = pd.DataFrame(Y_symbol.iloc[:, 0])
clustering_centroids_1 = pd.DataFrame(C.iloc[:, 0])

In [32]:
clustering_composition_1

,Clustering n°1
Cluster 1,"[A, ACN, AES, AIG, AKAM, ALLE, AME, AMP, AOS, ..."
Cluster 2,"[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ..."
Cluster 3,"[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ..."
Cluster 4,"[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ..."
Cluster 5,"[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ..."


In [26]:
len(clustering_composition_1.index)

5

In [36]:
clustering_centroids_1

,Clustering n°1
Cluster 1,"[-0.28931492711993867, -1.165199873872467, -1...."
Cluster 2,"[-0.12002057436487629, -0.09551460855918219, 0..."
Cluster 3,"[1.0818821358026836, -0.10293663224269473, -0...."
Cluster 4,"[-0.1280218009998571, 0.3798172936321998, 0.07..."
Cluster 5,"[1.5002065476511288, 1.0570143644422916, -0.31..."


In [37]:
cluster_1 = clustering_composition_1.iloc[0,0]
centroid_1 = clustering_centroids_1.iloc[0,0]

In [41]:
type(cluster_1)

list

In [42]:
weights = module1.cluster_weights(cluster_1, centroid_1, data)

In [48]:
cluster_data = data.loc[cluster_1]
cluster_data

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
A,-0.015976,-0.028044,-0.014895,0.001812,-0.026984,0.012966,0.011247,0.011503,-0.029123,0.004433,...,-0.007247,-0.006143,-0.003346,-0.003302,0.031605,-0.008461,0.011985,-0.023170,-0.006208,0.023503
ACN,-0.018130,-0.011093,-0.016852,-0.017544,-0.022246,0.026019,0.015041,-0.001748,-0.045574,-0.010361,...,-0.004245,0.004643,-0.014244,0.005310,0.013890,0.004935,0.007110,-0.009112,-0.004608,0.048720
AES,-0.001230,0.000818,-0.042792,-0.004294,-0.006042,-0.008241,0.014674,-0.000427,-0.007279,-0.005190,...,0.010788,-0.002182,-0.008639,0.017973,-0.004619,0.016640,-0.003489,-0.006753,0.006390,0.024148
AIG,0.003666,0.012793,-0.016025,-0.000844,0.018947,0.004734,0.003893,-0.001791,0.001294,0.015792,...,-0.009641,0.007599,-0.007826,0.006174,0.007254,0.006088,-0.004574,0.003298,0.008171,0.033678
AKAM,0.002898,-0.004692,-0.009995,-0.005082,-0.014995,0.028179,0.005481,0.004391,-0.011710,0.000888,...,-0.011168,0.016548,-0.009629,0.000110,0.005356,0.001742,0.007595,-0.002727,-0.002633,0.044183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,-0.021459,0.003413,-0.025137,-0.000248,0.000248,0.001745,0.011358,0.007826,0.009648,-0.009714,...,-0.011163,0.016056,-0.009561,-0.005916,0.001859,0.001557,0.004964,-0.024997,0.013951,0.030733
XRAY,0.020601,0.005085,-0.020757,-0.003551,-0.000177,-0.016278,0.015926,-0.028961,-0.015800,0.006883,...,-0.024214,0.022740,-0.003847,-0.009109,-0.007871,0.020216,0.017626,-0.024645,-0.005282,0.007297
XYL,-0.028007,0.019144,-0.015668,-0.005229,-0.004754,-0.002622,0.013430,-0.007461,-0.016641,-0.014773,...,-0.007092,0.002712,0.005609,0.002145,0.000178,0.003538,-0.002112,-0.025669,0.002369,0.024234
ZBH,0.013880,0.011876,-0.008349,-0.015088,0.007441,-0.003461,0.006329,-0.019556,0.014868,0.004169,...,-0.006097,-0.005603,0.004216,0.009739,0.007048,0.013995,0.006396,-0.003986,0.015583,0.005176


In [51]:
array_cluster_data = cluster_data.to_numpy()
array_weights_gaussian = weights_L2.to_numpy()

array_weights_L2 = weights_L2.to_numpy().T
result = np.sum(array_cluster_data * array_weights_L2, axis=0)

In [54]:
result_df = pd.DataFrame(result, columns=[clustering_composition_1.index[0]])

In [57]:
result_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,220,221,222,223,224,225,226,227,228,229
Cluster 1,-0.004038,0.010007,-0.016317,-0.000451,-0.00386,-0.003312,0.00697,-0.001798,-0.005376,0.002847,...,-0.007472,0.008684,-0.005912,0.00315,0.008354,0.00514,0.002459,-0.011981,0.003715,0.023854


In [102]:
micro_portfolio_return[B.index[0]] = D.transpose()

In [ ]:
df = df.append(, ignore_index=True)

In [27]:
data.loc[list_1]

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
AAPL,0.023234,-0.016173,-0.026459,-0.004061,-0.004173,0.018227,0.015890,-0.003356,-0.020635,0.010046,...,-0.002283,0.028876,-0.006719,-0.014422,0.013744,0.010781,-0.001349,-0.006359,-0.021860,0.045822
GOOGL,-0.000438,-0.006877,-0.047104,0.005452,-0.008202,0.026241,0.012450,0.001985,-0.021084,0.017367,...,0.009657,0.014233,-0.013660,-0.017523,0.011399,0.014012,-0.007971,-0.010254,-0.005657,0.063041
TSLA,0.044335,-0.034170,-0.052393,-0.011486,-0.050701,0.056494,0.010132,0.025053,-0.072450,0.028734,...,-0.024259,-0.004304,-0.026614,-0.046442,0.007562,0.053997,-0.011959,0.016314,-0.022744,0.065093
JPM,0.011444,0.021197,-0.024368,-0.008363,0.008954,-0.010405,-0.004587,-0.000119,-0.000654,-0.018014,...,0.000902,0.004083,-0.005440,-0.005173,0.007731,0.011348,0.001903,-0.012721,0.014011,0.015241
AVGO,-0.004513,0.012056,-0.042937,-0.004901,-0.026615,0.020000,0.010521,-0.008820,-0.049460,0.000990,...,-0.009704,0.013981,-0.003314,0.008735,0.013447,0.006184,-0.001471,-0.005785,-0.003083,0.050088
LLY,-0.009814,-0.008175,-0.028405,-0.002965,-0.000924,0.007409,0.002787,0.011871,-0.017470,-0.024401,...,0.008337,0.022028,-0.002430,0.002162,-0.003099,0.001051,0.006455,0.006363,-0.001839,0.016329
CVX,0.015549,0.012263,-0.000164,-0.002916,0.011908,-0.000320,0.016626,-0.005170,-0.002678,0.017207,...,-0.008344,0.014114,0.011155,0.008366,0.011579,0.005606,-0.006457,-0.010707,0.004263,0.002513
MRK,0.003780,0.007953,0.020751,0.000508,0.018096,0.022467,-0.009141,0.001479,0.004807,-0.002700,...,0.003509,0.018746,0.015470,0.005793,0.007229,0.000656,0.008783,0.012246,0.007748,0.012979
KO,0.008127,0.009332,0.011582,0.000000,0.000829,-0.001984,0.000331,0.000165,0.005929,0.007357,...,-0.004451,0.008602,0.002456,0.010851,-0.001923,0.002238,-0.000638,0.000160,-0.000640,0.025475
DHR,-0.035345,-0.018509,-0.013663,-0.002071,-0.023862,0.029572,0.000688,0.000000,-0.042731,0.011763,...,-0.014324,-0.014037,-0.016815,0.000923,-0.002287,-0.000152,0.007820,-0.014861,-0.003776,0.054144
